# Multi-Modal Defense

In [ ]:
import os
import json
import numpy as np
from keras.utils import to_categorical
from keras.utils import pad_sequences
import statistics


_docEvents = 'mousedown mouseup mousemove mouseover mouseout mousewheel wheel'
_docEvents += ' touchstart touchend touchmove deviceorientation keydown keyup keypress'
_docEvents += ' click dblclick scroll change select submit reset contextmenu cut copy paste'
_winEvents = 'load unload beforeunload blur focus resize error abort online offline'
_winEvents += ' storage popstate hashchange pagehide pageshow message beforeprint afterprint'

events = _docEvents.split() + _winEvents.split()

labels = {
        'human': 0,
        'gremlins': 1,
}

In [ ]:
# Load the Preprocessed Data
import pickle
pos_dict = {}

with open('data/pos_dict.pickle', 'rb') as handle:
    pos_dict = pickle.load(handle)


# Uni-Modality Version

# single_modality_pos_dict = {}
# mouse_indices = {}

# for c, values in pos_dict['event'].items():
#     mouse_indices[c] = []
#     for i, v in enumerate(values):
#         if v == 2:
#             mouse_indices[c].append(i)
# print(len(mouse_indices))

# for attr, v in pos_dict.items():
#     single_modality_pos_dict[attr] = {}  
#     for c, values in v.items():
#         single_modality_pos_dict[attr][c] = []
#         for i in mouse_indices[c]:
#             single_modality_pos_dict[attr][c].append(values[i])

# pos_dict = single_modality_pos_dict

In [ ]:
print(pos_dict.keys())
for k in list(labels.keys()):
    print(k, len(pos_dict['x'][k]))

In [ ]:
# Calculate one_hot data

from sklearn.preprocessing import LabelEncoder
import time

def get_timestamp_ms(t):
    return time.mktime(t.timetuple()) * 1000

n = len(pos_dict['x'])
n_x=0
n_y=0
n_action=0
pos_dict['x_one_hot']={}
pos_dict['y_one_hot']={}
pos_dict['action_encoded']={}
pos_dict['action_one_hot']={}
pos_dict['time_diff']={}

encoder = LabelEncoder()
all_unique_labels = set()
for k, label in labels.items():
    all_unique_labels.update(pos_dict['event'][k])
encoder.fit(list(all_unique_labels))

for k, label in labels.items():
    pos_dict['action_encoded'][k]=encoder.transform(pos_dict['event'][k])

for k, label in labels.items():
    n_x=np.max([n_x,len(set(pos_dict['dir_X'][k]))])
    n_y=np.max([n_y,len(set(pos_dict['dir_Y'][k]))])

n_action = len(all_unique_labels)


for k, label in labels.items():
    pos_dict['x_one_hot'][k] = to_categorical(pos_dict['dir_X'][k], num_classes=n_x)
    pos_dict['y_one_hot'][k] = to_categorical(pos_dict['dir_Y'][k], num_classes=n_y)
    pos_dict['action_one_hot'][k] = to_categorical(pos_dict['action_encoded'][k]-1, num_classes=n_action)

    
for k, label in labels.items():
    pos_dict['time_diff'][k]=[]
    for i in range(len(pos_dict['action'][k])-1):
#         if i%10000== 0:
#             print('{}/{}'.format(i, len(pos_dict['action'][k])))
        pos_dict['time_diff'][k]=np.append(pos_dict['time_diff'][k],get_timestamp_ms(pos_dict['t'][k][i+1])-get_timestamp_ms(pos_dict['t'][k][i]))




In [ ]:
from sklearn.utils import shuffle
import time
import numpy as np
import random


mousemove_index = 2

def get_train_data(time_steps, data_size=5000, train_index_limit=15000, multiModal=True):
    X, y, T = list(), list(), list()
    es = []
    for k, label in labels.items():
        n = train_index_limit if len(pos_dict['x'][k]) > train_index_limit else len(pos_dict['x'][k])
        counter=0
        while counter < data_size:
            x_seq = []
            counter+=1
            seq_start_i = random.randint(0, n - time_steps - 1)
            if multiModal:
                x_seq = [
                    np.concatenate([pos_dict['action_one_hot'][k][seq_i], pos_dict['x_one_hot'][k][seq_i],\
                                    pos_dict['y_one_hot'][k][seq_i],[pos_dict['time_diff'][k][seq_i]]])
                    for seq_i in range(seq_start_i, seq_start_i + time_steps)
                ]
            else:
                seq_i = seq_start_i
                while len(x_seq) < time_steps:
                    if pos_dict['event'][k][seq_i] == mousemove_index:
                        concat = np.concatenate([pos_dict['action_one_hot'][k][seq_i], pos_dict['x_one_hot'][k][seq_i],\
                                    pos_dict['y_one_hot'][k][seq_i],[pos_dict['time_diff'][k][seq_i]]])
                        x_seq.append(concat)
                    seq_i += 1
                    
            X.append(np.array(x_seq))
            y.append(to_categorical(label, len(set(labels.values()))))        
        
    X, y = np.array(X), np.array(y)
    print(X.shape, y.shape)
    X, y = shuffle(X, y, random_state=0)

    return X, y

def get_test_data(time_steps, time_window_ms, train_index_limit=15000, test_size=200, multiModal=True):
    X, y, T = list(), list(), list()
    es = []

    for k, label in labels.items():
        
        n = len(pos_dict['x'][k])
        try_counter = 0
        start_index = train_index_limit if len(pos_dict['x'][k]) > train_index_limit else int(len(pos_dict['x'][k])*0.7)

        for i in range(test_size):
            diff=0
            stop_flag = 0
            human_counter = 0
            while diff < time_window_ms:
                
                try_counter += 1
                stop_flag +=1 
                if stop_flag == 100:
                    break
                seq_start_i = random.randint(start_index, n-100)
                x_seq = list()
                for seq_i in range(seq_start_i, len(pos_dict['x'][k])-1):
                    if seq_start_i > n:
                        break
                    if seq_i - seq_start_i == time_steps:
                        break
                        
                    human_counter += 1

                    diff = pos_dict['t'][k][seq_i] - pos_dict['t'][k][seq_start_i]
                    diff = int(diff.total_seconds() * 1000.0)
                    if diff > time_window_ms:
                        diff = pos_dict['t'][k][seq_i-1] - pos_dict['t'][k][seq_start_i]
                        diff = int(diff.total_seconds() * 1000.0)
                        break
                    
                    x_seq.append(np.concatenate([pos_dict['action_one_hot'][k][seq_i], pos_dict['x_one_hot'][k][seq_i],pos_dict['y_one_hot'][k][seq_i],[pos_dict['time_diff'][k][seq_i]]]))
            
            X.append(x_seq)
            y.append(to_categorical(label, len(set(labels.values()))))
            T.append(diff)
    X, y, T = np.array(X), np.array(y), np.array(T)
    print(X.shape, y.shape, T.shape)
    X, y, T = shuffle(X, y, T, random_state=0)

    return X, y, T



In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Bidirectional
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from collections import deque



# n_features = len(events)+3
n_features = n_x+n_y+n_action+1

print(n_features)
# n_features = 3
n_steps = 50


def train(n_steps, epochs=100, batch_size=100):
    print('Train with Size:', n_steps)
    X, y = get_train_data(n_steps)

    print(X.shape, y.shape)
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    model = Sequential()

    return_sequences=True

    model.add(LSTM(300, return_sequences=return_sequences, activation='tanh', input_shape=(None, n_features)))
    # model.add(Dropout(0.2))
    model.add(LSTM(200, return_sequences=return_sequences, activation='tanh'))
    model.add(LSTM(100, activation='tanh'))
    model.add(Dense(len(set(labels.values())), activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

def test(model, assurance_threshold=8, test_size=200, train_index_limit=15000):
    print('TEST ---------- Size:', n_steps)
    time_to_detect = {}
    confusion_matrix = {}
    n_labels = len(list(labels.values()))
    for k, label in labels.items():
        n = len(pos_dict['x'][k])
        start_index = train_index_limit if len(pos_dict['x'][k]) > train_index_limit else int(len(pos_dict['x'][k])*0.7)
        
        time_to_detect[k] = []
        confusion_matrix[k] = np.zeros((n_labels, n_labels))
        for i in range(test_size):
            if i%(test_size/2) == 0:
                print(k, f"{i}/{test_size}")
            queue = deque([None] * assurance_threshold)
            # print(n, n_steps)
            
            seq_start_i = random.randint(start_index, n-100)
            
            # diff = pos_dict['t'][k][seq_start_i+80] - pos_dict['t'][k][seq_start_i]
            # diff = int(diff.total_seconds() * 1000.0)
            # while diff > 100000:
            #     seq_start_i = random.randint(start_index, n-100)
            #     diff = pos_dict['t'][k][seq_start_i+100] - pos_dict['t'][k][seq_start_i]
            #     diff = int(diff.total_seconds() * 1000.0)
            
            x_seq = list()
            
            for seq_i in range(seq_start_i, len(pos_dict['x'][k])-1):
                if seq_start_i > n:
                    break
                if seq_i - seq_start_i == 200:
                    break

                x_seq.append(np.concatenate([pos_dict['action_one_hot'][k][seq_i], pos_dict['x_one_hot'][k][seq_i],pos_dict['y_one_hot'][k][seq_i],[pos_dict['time_diff'][k][seq_i]]]))                
                y_predict = model.predict(np.array(x_seq).reshape((1, np.array(x_seq).shape[0], np.array(x_seq).shape[1])), verbose=0)
#                     true_label = to_categorical(label, len(set(labels.values())))
                
                true_label = label
                pred_label = np.argmax(y_predict[0])
        
                queue.popleft()
                queue.append(pred_label)
                
                if all(i == list(queue)[0] for i in list(queue)):
                    pred_label =  list(queue)[0]
                    diff = pos_dict['t'][k][seq_i] - pos_dict['t'][k][seq_start_i]
                    diff = int(diff.total_seconds() * 1000.0)
                    
                    confusion_matrix[k][true_label][pred_label] += 1
                    
                    correct = 0
                    if pred_label == true_label:
                        correct = 1
                    time_to_detect[k].append((diff, seq_i-seq_start_i, correct))
                    break

                
#     print(time_to_detect)
                    
    return time_to_detect, confusion_matrix

In [ ]:
import keras

res = {}
steps = [40, 50, 60]
models = {}
epochs = [50]
for step in steps:
    res[step] = {}
    models[step] = {}
    for epoch in epochs:
        model = train(step, epoch)
        # model.save()
        models[step][epoch] = model

In [ ]:
import copy

results = {}
ts = list(range(2, 20))
confusions = {}
for t in ts:
    res, confusion = test(model, assurance_threshold=t, test_size=200)
    confusions[t] = confusion
    results[t] = copy.deepcopy(res)
    print('------- Threshold', t)
    accs = []
    ttds = []
    etds = []
    for label, triples in res.items():
        acc = sum([i[2] for i in triples])/len(triples)
        accs.append(acc)
        ttd = sum([i[0] for i in triples])/len(triples)
        ttds.append(ttd)
        etd = sum([i[1] for i in triples])/len(triples)
        etds.append(etd)
        if label == 'random_mouse_bot':
            filler = '\t'
        else:
            filler = '\t\t'
        print(label, filler, f'ACC: {acc} \t TTD: {ttd} \t ETD: {etd}')
    
    acc_avg = round(sum(accs)/len(accs), 2)
    ttd_avg = round(sum(ttds)/len(ttds), 2)
    etd_avg = round(sum(etds)/len(etds), 2)

    print(f'\t \t \t acc_avg: {acc_avg}  \t ttd_avg: {ttd_avg} \t etd_avg: {etd_avg}')

# Brute Force Attack

In [ ]:
n_features = n_x+n_y+n_action+1
n_steps = 10 # Sequence length
mousemove_index = 2
train_X, train_y = get_train_data(n_steps)

In [ ]:

# Use PyTorch for easier attack implementation

import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import torch

epochs=20
batch_size=100
device = 'cuda:0'

class LSTMModel(nn.Module):
    def __init__(self, n_features, n_classes, return_sequences=True):
        super(LSTMModel, self).__init__()
        self.return_sequences = return_sequences

        self.lstm1 = nn.LSTM(n_features, 300, batch_first=True)
        self.lstm2 = nn.LSTM(300, 200, batch_first=True)
        self.lstm3 = nn.LSTM(200, 100, batch_first=True)

        # The final layer's size must be equal to the number of classes
        self.fc = nn.Linear(100, n_classes)
        
        # Sigmoid activation for binary classification or softmax for multiclass
        # self.activation = nn.Sigmoid() if n_classes == 2 else nn.Softmax(dim=1)
        
    def forward(self, x):
        # Assuming x is of shape (batch, seq, feature)
        x, _ = self.lstm1(x)

        if self.return_sequences:
            x, _ = self.lstm2(x)
        else:
            # Only pass the last output to the next LSTM if not return_sequences
            x, (h_n, c_n) = self.lstm2(x)
            x = h_n[-1]
            
        x, _ = self.lstm3(x)

        # If return_sequences is True, we need to only take the output at the last timestep
        if self.return_sequences:
            x = x[:, -1, :]
        
        x = self.fc(x)
        # x = self.activation(x)
        return x

# Model instantiation
n_features = train_X.shape[2]  # Assuming X is (batch, seq, feature)
n_classes = train_y.shape[1]

model = LSTMModel(n_features=n_features, n_classes=n_classes, return_sequences=False).to(device)


# Convert data to PyTorch tensors
train_X_tensor = torch.tensor(train_X, dtype=torch.float32,device=device)
train_y_tensor = torch.tensor(train_y, dtype=torch.long,device=device)



if len(train_y_tensor.shape) > 1 and train_y_tensor.shape[1] > 1:
    _, train_y_tensor = train_y_tensor.max(dim=1)

# Create a dataset and dataloader for batching
dataset = TensorDataset(train_X_tensor, train_y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Pytorch model train

from tqdm import tqdm
import matplotlib.pyplot as plt

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # It applies softmax internally
optimizer = Adam(model.parameters(),lr=1e-3)

# Training loop
train_losses = []

for epoch in range(epochs):
    # Initialize loss for the epoch
    epoch_loss = 0.0
    # Loop over the data in batches using the dataloader
    for batch_x, batch_y in tqdm(dataloader, desc=f'Epoch {epoch+1}/{epochs}'):
        optimizer.zero_grad()  # Clear gradients for the next train
        output = model(batch_x)  # Forward pass
        loss = criterion(output, batch_y)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        epoch_loss += loss.item() * batch_x.size(0)  # Multiply by batch size since loss is averaged

    # Calculate the average loss for this epoch
    epoch_loss /= len(dataloader.dataset)
    train_losses.append(epoch_loss)  # Append the average loss for the epoch to the list
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')

plt.figure(figsize=(10, 5))  
plt.plot(train_losses, label='Training Loss')
plt.title('Train Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Test the model

from collections import deque
import random
import numpy as np
import torch

def test(model, assurance_threshold=8, test_size=200, train_index_limit=15000):
    model.eval()  # Set the model to evaluation mode
    test_set = []
    test_set_whole = []
    with torch.no_grad():  # Inference without tracking gradients
        print('TEST ----------')
        time_to_detect = {}
        confusion_matrix = {}
        n_labels = len(set(labels.values()))

        for k, label in labels.items():
            n = len(pos_dict['x'][k])
            start_index = train_index_limit if len(pos_dict['x'][k]) > train_index_limit else int(n * 0.7)
            
            time_to_detect[k] = []
            confusion_matrix[k] = np.zeros((n_labels, n_labels))

            for i in range(test_size):
                if i % (test_size // 2) == 0:
                    print(k, f"{i}/{test_size}")
                queue = deque([None] * assurance_threshold)
                
                seq_start_i = random.randint(start_index, n - 100)
                
                x_seq = []

                for seq_i in range(seq_start_i, min(n - 1, seq_start_i + 200)):
                    x_seq.append(np.concatenate([
                        pos_dict['action_one_hot'][k][seq_i],
                        pos_dict['x_one_hot'][k][seq_i],
                        pos_dict['y_one_hot'][k][seq_i],
                        [pos_dict['time_diff'][k][seq_i]]
                    ]))

                    input_tensor = torch.tensor(np.array([x_seq]), dtype=torch.float32).to(device)  # Shape (1, seq_len, features)
                    y_predict = model(input_tensor).cpu().numpy()  # Inference and convert to numpy array
                    pred_label = np.argmax(y_predict, axis=1)[0]  # Get the predicted label
                    test_set.append((input_tensor,pred_label,label))
                    queue.popleft()
                    queue.append(pred_label)
                    
                    if all(v == queue[0] for v in queue):
                        pred_label = queue[0]
                        diff = (pos_dict['t'][k][seq_i] - pos_dict['t'][k][seq_start_i]).total_seconds() * 1000.0
                        
                        confusion_matrix[k][label][pred_label] += 1
                        
                        correct = int(pred_label == label)
                        time_to_detect[k].append((int(diff), seq_i - seq_start_i, correct))
                        break
                
                x = torch.tensor(np.array([x_seq]), dtype=torch.float32).to(device)
                y_pred = model(x).cpu().numpy()  # Inference and convert to numpy array
                y_label = np.argmax(y_predict, axis=1)[0]  # Get the predicted label
                test_set_whole.append((x,y_label,label))

        return time_to_detect, confusion_matrix, test_set, test_set_whole

import copy

results = {}
# ts = list(range(2, 20,4))
ts = [10]
confusions = {}
acc_avgs = []
test_sets = []
for t in ts:
    res, confusion,test_set, test_set_whole = test(model, assurance_threshold=t, test_size=200)
    test_sets.append(test_set_whole)

In [ ]:
test_set_ = test_sets[0] # diffferent lengths
print(len(test_set_))

data_label = [[] for _ in labels.keys()]
for data in test_set_:
    x,p_y,y = data
    data_label[y].append((x,p_y))

index = 5 # Random index to check dimensions of samples
label = 1

input_sequence = data_label[label][index][0] # get the actual data not the label (2nd item)
# input_sequence = torch.split(input_sequence, 5, dim=1)[3]
print(input_sequence.shape)

In [ ]:
# Brute Force Attack based on the test samples

import itertools


def brute_force(min_time, max_time, source_label=1, seq_len=3, test_index=0):
    print('Source = ', source_label)
    test_set_ = test_sets[0] # diffferent lengths
#     print(len(test_set_))

    data_label = [[] for _ in labels.keys()]
    for data in test_set_:
        x,p_y,y = data
        data_label[y].append((x,p_y))

    index = test_index
    
    label = source_label

        
    if data_label[label][index][0].shape[1] < seq_len:
        print('No seq long enough...')
        return 
#     print(data_label[label][index])
    input_sequence = data_label[label][index][0] # get the actual data not the label (2nd item)
    input_sequence = torch.split(input_sequence, seq_len, dim=1)[0]
    print(input_sequence.shape)
    
    extensions = []
    seq_len = seq_len
    count = 0 
    for c1 in itertools.combinations_with_replacement(list(range(5)), seq_len):
        for c2 in itertools.combinations_with_replacement(list(range(5)), seq_len):
            for t in itertools.combinations_with_replacement(list(range(min_time, max_time)), seq_len):
                count += 1
                extensions.append([c for c in c1] + [c for c in c2] + [c for c in t])

    global_index = 0

    n_batches = 500

    batch_size = int((count-(count%n_batches))/n_batches)


    successful_attacks = []
    for batch_index in range(n_batches):
        input_sequence_batch = input_sequence.repeat(batch_size, 1, 1)
    #     print(input_sequence_batch.shape)
        if batch_index%100 == 0:
            print(f'{batch_index}/{n_batches}')
        for batch_index in range(batch_size):
            for s in range(seq_len):
                input_sequence_batch[batch_index, s, -11:-6] = torch.nn.functional.one_hot(torch.tensor(extensions[global_index][0+s]), num_classes=5)
                input_sequence_batch[batch_index, s, -6:-1] = torch.nn.functional.one_hot(torch.tensor(extensions[global_index][seq_len+s]), num_classes=5)
                input_sequence_batch[batch_index, s, -1] = torch.tensor((extensions[global_index][2*seq_len+s])).float()

        global_index += 1

        input_sequence_batch = input_sequence_batch.to(device)
        target_label = torch.zeros(batch_size, dtype=torch.long, device=device)
        with torch.no_grad():
            final_output = model(input_sequence_batch)
            final_predictions = final_output.argmax(dim=1)

        # Check for successful attacks
        attack_success_indices = torch.where(final_predictions == target_label)[0]

        # successful_attacks = [(x_vals[i].item(), y_vals[i].item(), time_vals[i].item()) for i in attack_success_indices]
        successful_attacks.extend([input_sequence_batch[i] for i in attack_success_indices]) 
    
#     print(count, len(successful_attacks), len(successful_attacks[0]))
    return count, successful_attacks

In [ ]:
# Test and Time the attack for differnet settings
import time

res = {}
for t in range(15, 40, 3):
    for label in [1]:
        if t not in res:
            res[t] = {}
        
        if label not in res[t]:
            res[t][label] = {}
        for s in [3, 6, 10]:
            if s not in res[t][label]:
                res[t][label][s] = []
            else:
                continue
            start = time.time()
            print('----', t, s)
            count, successful_attacks = brute_force(t, t+1, source_label=label, seq_len=s)
            end = time.time()
            print('---- Time:', end - start)
            print("===== Res", count, len(successful_attacks), len(successful_attacks)/count*100)
            res[t][label][s] = [count, len(successful_attacks), len(successful_attacks)/count*100]